📥 1. Load & Split Dataset

In [20]:
import tensorflow as tf
from keras.datasets import fashion_mnist

# Load dataset
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# Normalisasi skala pixel: 0–255 → 0–1
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

# Bagi menjadi train dan validasi
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

🧪 2. Fungsi Preprocessing

In [21]:
def preprocess(X, y):
    X = tf.cast(X, tf.float32)
    X = tf.expand_dims(X, -1)  # ubah shape jadi (28, 28, 1)
    return X, y

🎨 3. (Opsional) Fungsi Augmentasi Gambar

In [22]:
def augment(X, y):
    X = tf.image.random_flip_left_right(X)
    X = tf.image.random_brightness(X, max_delta=0.1)
    return X, y

🧵 4. Membuat Pipeline tf.data

In [23]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = (
    train_dataset
    .shuffle(buffer_size=10000)
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .map(augment, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

In [24]:
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
valid_dataset = (
    valid_dataset
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

🏗️ 5. Bangun dan Latih Model

In [25]:
from keras import models, layers

model = models.Sequential([
    layers.Input(shape=[28, 28, 1]),
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model.fit(train_dataset, epochs=10, validation_data=valid_dataset)

Epoch 1/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - accuracy: 0.7557 - loss: 0.6849 - val_accuracy: 0.8718 - val_loss: 0.3503
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.8738 - loss: 0.3420 - val_accuracy: 0.8948 - val_loss: 0.2928
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.8928 - loss: 0.2909 - val_accuracy: 0.9062 - val_loss: 0.2523
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9043 - loss: 0.2581 - val_accuracy: 0.9128 - val_loss: 0.2425
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9138 - loss: 0.2362 - val_accuracy: 0.9118 - val_loss: 0.2433
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9185 - loss: 0.2186 - val_accuracy: 0.9124 - val_loss: 0.2374
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9247 - loss: 0.2038 - val_accuracy: 0.9070 - val_loss: 0.2488
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9300 - loss: 0.188

## ✅ Catatan:

* Kita tidak menggunakan `X_train` langsung, tetapi melalui `tf.data.Dataset`, agar efisien dan scalable.
* `.map()` digunakan untuk menerapkan fungsi preprocessing dan augmentasi.
* `.prefetch(tf.data.AUTOTUNE)` membantu mempercepat training dengan menjalankan pipeline dan model secara paralel.
* `.expand_dims()` penting agar data masuk ke CNN dalam format `(28, 28, 1)`.